In [ ]:
#Instalar as Dependências
pip install mysql-connector
pip install pynubank

In [ ]:
from pynubank import Nubank
import mysql.connector
import datetime

In [ ]:
nu = Nubank()
uuid, qr_code = nu.get_qr_code()
qr_code.print_ascii(invert=True)
input('Após escanear o QRCode pressione enter para continuar')
# Nesse momento será printado o QRCode no console
# Você precisa escanear pelo o seu app do celular
# Esse menu fica em NU > Perfil > Acesso pelo site

In [ ]:
# Somente após escanear o QRCode você pode chamar a linha abaixo
# Insira seu CPF e sua senha da sua conta do Nubank
nu.authenticate_with_qr_code('CPF', 'SENHA', uuid)
print(nu.get_account_balance())

In [ ]:
card_statements = nu.get_card_statements()

In [ ]:
mydb = mysql.connector.connect(
  host="HOST",
  user="USER",
  password="SENHA"
)
cursor = mydb.cursor()
cursor = mydb.cursor(buffered=True)

In [ ]:
#Cria a estrutura do banco
cursor.execute("CREATE DATABASE IF NOT EXISTS nubank;")
cursor.execute("USE nubank;")
cursor.execute( """CREATE TABLE IF NOT EXISTS `fatura_cartao` (
  `fatura_id` varchar(100) NOT NULL,
  `descricao` varchar(255) DEFAULT NULL,
  `categoria` varchar(100) DEFAULT NULL,
  `valor` decimal(13,2) DEFAULT NULL,
  `datetime` varchar(100),
  `origem` varchar(100) DEFAULT NULL,
  `titulo` varchar(100) DEFAULT NULL,
  `conta` varchar(100) DEFAULT NULL,
  `lat` varchar(30) DEFAULT NULL,
  `lon` varchar(30) DEFAULT NULL,
  `updated_at` timestamp NOT NULL DEFAULT current_timestamp() ON UPDATE current_timestamp(),
  `created_at` timestamp NOT NULL DEFAULT current_timestamp(),
  PRIMARY KEY (`fatura_id`)
);""")
cursor.execute( """CREATE TABLE IF NOT EXISTS `conta_corrente` (
  `id` varchar(255) NOT NULL,
  `operacao` varchar(50) DEFAULT NULL,
  `titulo` varchar(50) DEFAULT NULL,
  `detalhe` varchar(255) DEFAULT NULL,
  `data` varchar(100),
  `Valor` float DEFAULT NULL,
  `created_at` timestamp NOT NULL DEFAULT current_timestamp(),
  `updated_at` datetime DEFAULT current_timestamp() ON UPDATE current_timestamp(),
  PRIMARY KEY (`id`)
);""")


In [ ]:
#Faz a carga das transações do Cartão de Crédito
for idx,compra in enumerate(card_statements):
    cursor.execute("SELECT * FROM fatura_cartao WHERE fatura_id = %s",(compra['id'], ))
    if(cursor.rowcount == 0):
        print('Linha',idx,' Inserida')
        details = compra.get("details",None)
        if(details != None):
            long = details.get("lon", None)
            lat = details.get("lat", None)
        cursor.execute("INSERT INTO fatura_cartao VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);",(compra['id'],compra['description'],compra['category'],compra['amount']/100,compra['time'],compra.get("source",None),compra['title'],compra.get("account",None),lat,long, datetime.datetime.now(), datetime.datetime.now()))
        

In [ ]:
# Lista de dicionários contendo todas as transações da conta
account_statements = nu.get_account_statements()

In [ ]:
#Faz a carga das transações da Conta Corrente
for idx,t in enumerate(account_statements):
    if t['__typename'] == "TransferInEvent" :
        tipo_operacao = "Entrada"
        valor = t['amount']
    elif t['__typename'] == "TransferOutEvent" :
        tipo_operacao = "Saída"
        if( t['amount']  != None):
          valor = t['amount'] * -1
    elif t['__typename'] == "TransferOutReversalEvent" :   
        tipo_operacao = "Devolução"
        valor = t['amount']
    elif t['__typename'] == "DebitPurchaseEvent" :
        tipo_operacao = "Compra no Débito"
        if( t['amount']  != None):
          valor = t['amount'] * -1
    elif t['__typename'] == "BarcodePaymentEvent" :
        tipo_operacao = "Compra no Boleto"
        if( t['amount']  != None):
          valor = t['amount'] * -1       
    elif t['__typename'] == "DebitWithdrawalFeeEvent" :
        tipo_operacao = "Tarifa de Saque"
        if( t['amount']  != None):
          valor = t['amount'] * -1
    elif t['__typename'] == "DebitWithdrawalEvent" :
        tipo_operacao = "Saque"
        if( t['amount']  != None):
          valor = t['amount'] * -1
    elif t['__typename'] == "BillPaymentEvent" :
        tipo_operacao = "Pagamento da Fatura do Cartão de Crédito"
        valor = float(t['detail'][19:].replace(".", "").replace(",", ".")) * -1
    else:
        valor = t['amount']   
    cursor.execute("SELECT * FROM conta_corrente WHERE id = %s",(t['id'], )) 
    if(cursor.rowcount == 0):
        print('Linha',idx,' Inserida')
        cursor.execute("INSERT INTO conta_corrente VALUES (%s,%s,%s,%s,%s,%s,%s,%s);",(t['id'],tipo_operacao,t['title'],t['detail'],t['postDate'],valor,datetime.datetime.now(), datetime.datetime.now()))

In [ ]:
#Faz o commit de todas transações no Banco de Dados
mydb.commit()